In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data import Envelope
from system import AccountingSystem
from utils import dollars

In [3]:
sys = AccountingSystem()
sys.load()

In [4]:
import panel as pn

pn.extension("tabulator", notifications=True)
pn.widgets.Tabulator.theme = "materialize"

NotificationArea()

# Accounts Table

In [5]:
from bokeh.models.widgets.tables import BooleanFormatter, NumberFormatter, SelectEditor

In [6]:
# from panel.viewable import Viewer

# class EditableAccountsTable(Viewer):
#     def __init__(self):
#         self.setup_widgets()

#     def setup_widgets(self):
#         self.add_account_button = pn.widgets.Button(name="Add", button_type="primary")
#         self.accounts_table = pn.widgets.Tabulator(
#             sys.data.accounts,
#             formatters={
#                 "track": BooleanFormatter(),
#                 "amount": NumberFormatter(format="$0,0.00", font_style="bold"),
#             },
#             editors={
#                 "track": SelectEditor(options=["True", "False"]),
#                 "amount": {"type": "number", "verticalNavigation": "table"},
#             },
#         )
#         self.layout = pn.Column(self.accounts_table, self.add_account_button)

In [7]:
add_account_button = pn.widgets.Button(name="Add", button_type="primary")
accounts_table = pn.widgets.Tabulator(
    sys.data.accounts,
    formatters={
        "track": BooleanFormatter(),
        "amount": NumberFormatter(format="$0,0.00", font_style="bold"),
    },
    editors={
        # "track": SelectEditor(options=["True", "False"]),
        "track": None,
        "amount": {"type": "number", "verticalNavigation": "table"},
    },
    selectable=False,
)


def check_toggle_track(event):
    if event.column == "track":
        current_value = sys.data.accounts.loc[event.row, "track"]
        sys.data.accounts.loc[event.row, "track"] = not current_value
        accounts_table.value = accounts_table.value = sys.data.accounts


accounts_table.on_click(check_toggle_track)


def add_account(event):
    sys.data.accounts = sys.data.accounts.append(
        dict(name="name", amount=0.0, track=False), ignore_index=True
    )
    accounts_table.value = sys.data.accounts


add_account_button.on_click(add_account)

Watcher(inst=Button(button_type='primary', name='Add'), cls=<class 'panel.widgets.button.Button'>, fn=<function add_account at 0x7f8daa5f5e10>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [8]:
accounts_editor = pn.Column(accounts_table, add_account_button)
accounts_editor

Column
    [0] Tabulator(editors={'track': None, ...}, formatters={'track': BooleanFormatter...}, selectable=False, value=Empty DataFrame
Columns: [...)
    [1] Button(button_type='primary', name='Add')

In [9]:
sys.data.accounts

,name,amount,track


In [ ]:
sys.data.envelopes

In [23]:
sys.take_envelopes_snapshot()
sys.take_accounts_snapshot()

In [ ]:
sys.data.envelope_history

In [ ]:
sys.data.account_history

In [ ]:
sys.grab_accounts()
sys.data.transfers

In [ ]:
sys.data.envelopes

In [ ]:
accounts_editor

In [ ]:
accounts_editor.show()

In [ ]:
def calculate_total():
    return sys.data.accounts[sys.data.accounts.track].amount.sum()


pn.indicators.Number(name="Total", value=calculate_total(), default_color="white")

# Envelopes

In [110]:
import param
from panel.viewable import Viewer

envelope_colors = {
    "Cost": "#992211",
    "Emergency": "#775533",
    "Save": "#228844",
    "Spend": "#224488",
    "Internal/Unaccounted": "#449988",
    "Internal/Expense Queue": "#BB7711",
    "Internal/Staged Expense": "#AA2277",
}


class EnvelopePanel(Viewer):

    css = """
    .bk .envelope-card {
        background: #119933;
        color: white !important;
        border-radius: 20px !important;
        margin: 10px;
        line-height: .5;
    }
    
    .bk .envelope-card h1,
    .bk .envelope-card h3 {
        padding: 0px;
        margin: 0px;
    }
    .bk .envelope-card h3 {
        text-align: left;
        margin-left: -10px;
        margin-top: -5px;
        color: #FFFFFF88;
    }
    .bk .envelope-card small {
        font-size: 12px;
        color: #FFFFFFAA;
    }
    
    .bk .card-button {
        display: none !important;
    }
    
    .bk .curvy {
        border-radius: 20px !important;
    }
    
    
    .bk .arrow-toggle .bk-btn,
    .bk .arrow-toggle .bk-btn .bk-active {
        background-color: transparent;
        color: white;
        border: none;
        border-bottom: 1px solid;
        box-shadow: none;
        font-size: 20pt;
    }
    .bk .arrow-toggle .bk-btn:hover {
        color: yellow;
    }
    
    .bk .nice-select .bk-input {
        background-color: transparent;
        color: white;
        margin-right: 20px;
        padding: 0;
    }
    
    .area {
        background-color: rgba(255, 255, 255, .3);
        /*padding: 5px;*/
        border-radius: 20px;
    }
    """

    name = param.String(default="str")
    value = param.Number()
    display_graph_in_header = param.Boolean(False)

    def __init__(self, envelope: Envelope, system=None):
        self.envelope = envelope
        self.amount_pane = pn.pane.HTML("")
        
        self.system = system
        envelope_names = []
        if self.system is not None:
            envelope_names = [env.name for env in self.system.data.envelopes]
            envelope_names.remove(envelope.name)
        
        self.transfer_this_amount = pn.pane.HTML("")
        self.transfer_direction = pn.widgets.Toggle(name='>', css_classes=["arrow-toggle"], width=50, height=40)
        self.transfer_amount = pn.widgets.FloatInput(name='Amount', css_classes=["nice-select"], width=100)
        self.transfer_that_amount = pn.pane.HTML("")
        self.transfer_that_account = pn.widgets.Select(name='Envelope', options=envelope_names, css_classes=["nice-select"], width=150)
        
        

        self.layout = pn.Card(
            pn.Column(
                pn.Row(
                    pn.pane.HTML("<h2>Transfer</h2>"), 
                    self.transfer_direction,
                    self.transfer_that_account, 
                ),
                pn.Row(
                    self.transfer_this_amount,
                    self.transfer_amount,
                    self.transfer_that_amount
                ),
                css_classes=['area']
            ),
            header=self.collapsed_header_layout(),
            background=envelope_colors[self.envelope.category],
            header_color="white",
            header_background=envelope_colors[self.envelope.category],
            header_css_classes=["curvy"],
            css_classes=["envelope-card", "curvy"],
        )
        self.layout.collapsed = True
        self.refresh_header()
        super().__init__()

    def collapsed_header_layout(self):
        return pn.FlexBox(pn.Column(self.amount_pane))

    def expanded_header_layout(self):
        return pn.FlexBox(
            pn.Row(pn.Column(self.amount_pane), pn.Column(
                pn.pane.Str("YO DAWG")
            ))
        )

    def refresh_header(self):
        amount = f"<h1>{dollars(self.envelope.amount)}"
        if self.envelope.goal is not None:
            amount += (
                f"<small>/{dollars(self.envelope.goal, dollar_sign=False)}</small>"
            )
        amount += "</h1>"
        self.amount_pane.object = f"<h3>{self.envelope.name}</h3>{amount}"
        
        self.refresh_transfer_amounts()
        
    def refresh_transfer_amounts(self):
        self.update_transfer_values()

    @param.depends("layout.collapsed", watch=True)
    def header_clicked(self):
        if self.layout.collapsed:
            self.layout.header = self.collapsed_header_layout()
        else:
            self.layout.header = self.expanded_header_layout()
            
    @param.depends("transfer_direction.value", watch=True)
    def update_transfer_dir(self):
        if self.transfer_direction.value:
            self.transfer_direction.name = "<"
        else:
            self.transfer_direction.name = ">"
            
    @param.depends("transfer_amount.value", "transfer_that_account.value", "transfer_direction.value", watch=True)
    def update_transfer_values(self):
        this_amt = self.envelope.amount
        
        if self.system is not None:
            opposing_env = self.system.data.get_envelope_by_name(self.transfer_that_account.value)
            opposing_amt = opposing_env.amount
            if self.transfer_direction.value:
                opposing_amt -= self.transfer_amount.value
                this_amt += self.transfer_amount.value
            else:
                opposing_amt += self.transfer_amount.value
                this_amt -= self.transfer_amount.value
            self.transfer_that_amount.object = f"<p style='padding-left: 20px;'><b>{dollars(opposing_amt)}</b></p>"
            
        self.transfer_this_amount.object = f"<p style='padding-left: 20px;'><b>{dollars(this_amt)}</b></p>"
            

    def update(self):
        """Update visual layout from underlying envelope data."""
        pass

    def __panel__(self):
        return self.layout

In [111]:
pn.extension("tabulator", raw_css=[EnvelopePanel.css], notifications=True)

NotificationArea(position='top-right')

In [112]:
env = Envelope(name="Testing", category="Emergency", amount=123.45)

In [113]:
envelopes_example = pn.FlexBox(
    EnvelopePanel(
        Envelope(name="Cost", category="Cost", amount=123.45, goal=200.00, capped=True)
    ),
    EnvelopePanel(Envelope(name="Emergency", category="Emergency", amount=123.45)),
    EnvelopePanel(Envelope(name="Save", category="Save", amount=123.45)),
    EnvelopePanel(Envelope(name="Spend", category="Spend", amount=123.45)),
    EnvelopePanel(
        Envelope(name="Unaccounted", category="Internal/Unaccounted", amount=123.45)
    ),
    EnvelopePanel(
        Envelope(name="Expense Queue", category="Internal/Expense Queue", amount=123.45)
    ),
    EnvelopePanel(
        Envelope(
            name="Staged Expense", category="Internal/Staged Expense", amount=123.45
        )
    ),
)
envelopes_example

FlexBox(objects=[Card(background='#992211'...], sizing_mode='stretch_width')

In [114]:
# envelopes_example.show()

For normal envelopes, to do transfer:
Select from dropdown other account, then it shows: (note that the default other envelope should be Unaccounted)

    Env1 -> Env2
    amt

and the `->` is clickable to toggle transfer direction.



When click on the internal envelopes, rather than a normal envelope having related transactions etc, only have form related to that envelope:

* Unaccounted: (normal transfer), distribution plan list/button
* Expense Queue: filtered table to unaccounted expenses, so can directly handle them there 
* Staged expense: filtered table to expenses, possibly a clear all as well.

In [115]:
pn.state.notifications.position = "top-right"

# accounts_editor
envpanels = [EnvelopePanel(env, sys) for env in sys.data.envelopes]

envs = pn.FlexBox(*envpanels)

grab_button = pn.widgets.Button(name="Grab Accounts", button_type="default")


def grab(event):
    diff = sys.grab_accounts()
    for env in envpanels:
        env.refresh_header()
    if diff < 0.0:
        pn.state.notifications.warning(
            f"Negative amount ({dollars(diff, False)}) transferred to unaccounted envelope. Perhaps there are outstanding staged expenses?",
            duration=0,
        )
    elif diff == 0.0:
        pn.state.notifications.info(
            "Unaccounted envelope is up to date!", duration=5000
        )
    else:
        pn.state.notifications.success(
            f"{dollars(diff)} additional income added to unaccounted!", duration=5000
        )


grab_button.on_click(grab)

pn.Column(accounts_editor, grab_button, envs)

Column
    [0] Column
        [0] Tabulator(editors={'track': None, ...}, formatters={'track': BooleanFormatter...}, selectable=False, value=       name  a...)
        [1] Button(button_type='primary', clicks=1, name='Add')
    [1] Button(name='Grab Accounts')
    [2] FlexBox(objects=[Card(background='#449988'...], sizing_mode='stretch_width')

In [116]:
sys.data.transfers

,envelope_from,envelope_to,amount,type,description,date_entered,date_processed,accounted,tags
0,None,0,3.0,Income,,2022-08-17 21:33:21.628864,NaN,NaN,[accounts_grab]
1,None,0,3.0,Income,,2022-08-17 21:33:32.490301,NaN,NaN,[accounts_grab]
2,None,0,-1.0,Income,,2022-08-17 21:57:12.256791,NaN,NaN,[accounts_grab]
3,None,0,7.0,Income,,2022-08-17 21:57:22.394783,NaN,NaN,[accounts_grab]
